In [1]:
import os
os.getcwd()

'/home/ajana/Code/MLProjects/Heart-Disease-Detection/research'

In [2]:
os.chdir('../')

In [3]:
os.getcwd()

'/home/ajana/Code/MLProjects/Heart-Disease-Detection'

In [4]:
from heart.entity.artifact_entity import (DataValidationArtifact,DataTransformationArtifact,
                                        ModelTrainerArtifact,ModelEvaluationArtifact)
from heart.entity.config_entity import ModelEvaluationConfig
import os,sys
from heart.ml.metric.classification_metric import get_classification_score
from heart.ml.model.estimator import HeartModel
from heart.utils import save_object,load_object,write_yaml_file
from heart.ml.model.estimator import ModelResolver
from heart.constant.training_pipeline import TARGET_COLUMN
import pandas  as  pd

In [5]:
from heart.pipeline.training_pipeline import TrainingPipeline
from heart.entity.config_entity import TrainingPipelineConfig

In [6]:
training_config= TrainingPipelineConfig()
training_pipeline = TrainingPipeline(training_config=training_config)

In [7]:
data_ingestion_artifact = training_pipeline.start_data_ingestion()
data_validation_artifact = training_pipeline.start_data_validation(data_ingestion_artifact=data_ingestion_artifact)
data_transformation_artifact = training_pipeline.start_data_transformation(data_validation_artifact=data_validation_artifact)
model_trainer_artifact = training_pipeline.start_model_trainer(data_transformation_artifact)

In [8]:
model_resolver = ModelResolver()

In [9]:
model_resolver.transformer_dir_name

'transformed'

In [10]:
latest_dir_path = model_resolver.get_latest_dir_path()


In [11]:
if latest_dir_path==None:
    is_model_accepted=True
    model_evaluation_artifact = ModelEvaluationArtifact(
        is_model_accepted=is_model_accepted, 
        improved_accuracy=None, 
        best_model_path=None, 
        trained_model_path=model_trainer_artifact.trained_model_file_path, 
        train_model_metric_artifact=model_trainer_artifact.test_metric_artifact, 
        best_model_metric_artifact=None)
model_evaluation_artifact

ModelEvaluationArtifact(is_model_accepted=True, improved_accuracy=None, best_model_path=None, trained_model_path='artifact/01_04_2024_02_04_32/model_trainer/model/model.pkl', train_model_metric_artifact=ClassificationMetricArtifact(f1_score=0.9845201238390092, precision_score=0.9875776397515528, recall_score=0.9814814814814815), best_model_metric_artifact=None)

In [20]:
#Finding location of transformer and target encoder
transformer_path = model_resolver.get_latest_transformer_path()
model_path = model_resolver.get_latest_model_path()
target_encoder_path = model_resolver.get_latest_target_encoder_path()

Exception: Transformer is not available

In [12]:

# logging.info(f"Current trained objects of transformer, model and target encoder")
current_transformer = load_object(file_path=data_transformation_artifact.transformed_object_file_path) 
current_model = load_object(file_path=model_trainer_artifact.trained_model_file_path) 
current_target_encoder = load_object(file_path=data_transformation_artifact.target_encoder_path)

# valid_train_file_path = self.data_validation_artifact.valid_train_file_path
valid_test_file_path = data_validation_artifact.valid_test_file_path

#valid train and test file dataframe
# train_df = pd.read_csv(valid_train_file_path)
test_df = pd.read_csv(valid_test_file_path)


# y_true = test_df[TARGET_COLUMN]
# y_true = target_encoder.transform(y_true) 
# test_df.drop(TARGET_COLUMN,axis=1,inplace=True)


In [13]:
test_df

,age,gender,impluse,pressurehight,pressurelow,glucose,kcm,troponin,class
0,64,0,68,91,61,119.0,2.97,1.530,positive
1,43,0,79,89,57,98.0,2.46,0.006,negative
2,82,1,62,126,65,137.0,39.53,0.006,positive
3,34,0,81,125,69,152.0,2.07,0.013,negative
4,41,1,86,113,68,100.0,4.58,0.003,negative
...,...,...,...,...,...,...,...,...,...
259,66,1,66,160,83,103.0,71.77,0.015,positive
260,79,1,89,140,100,157.0,4.74,0.191,positive
261,64,1,65,155,75,107.0,1.30,0.004,negative
262,72,0,75,160,70,130.0,8.54,0.015,positive


In [14]:
y_true = test_df[TARGET_COLUMN]
y_true = current_target_encoder.transform(y_true) 
# test_df.drop(TARGET_COLUMN,axis=1,inplace=True)
y_true

array([1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1,
       1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0,
       0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1,
       0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1,
       0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1])

In [18]:
input_feature_name = current_transformer.feature_names_in_

In [24]:
input_arr = test_df[input_feature_name]
input_arr.shape

(264, 8)

In [25]:
y_pred = current_model.predict(input_arr)

In [23]:
current_model

In [26]:
get_classification_score(y_true,y_pred)

ClassificationMetricArtifact(f1_score=0.9845201238390092, precision_score=0.9875776397515528, recall_score=0.9814814814814815)

In [27]:
current_target_encoder.inverse_transform(y_pred[:5])

array(['positive', 'negative', 'positive', 'negative', 'negative'],
      dtype=object)

In [5]:
timestamps =  list(map(int,os.listdir("saved_models")))
latest_timestamp = max(timestamps)
latest_model_path= os.path.join("saved_models",f"{latest_timestamp}","model","model.pkl")

In [6]:
latest_model_path

'saved_models/1704372023/model/model.pkl'